<a href="https://colab.research.google.com/github/ksj1999/DataMining_Group_Project_1/blob/main/GP1_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparing Dataset & Library

In [ ]:
# !pip install statsmodels

You should consider upgrading via the 'c:\users\kwon\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
import statsmodels.api as sm
import pandas as pd
import numpy as np

### Clean column names

In [ ]:
concrete_df = pd.read_csv("Concrete_Data.csv")
concrete_df.columns = concrete_df.columns.str.split('(').str[0].str.strip()
concrete_df.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


### Check for null

In [ ]:
concrete_df.isna().sum()

Cement                           0
Blast Furnace Slag               0
Fly Ash                          0
Water                            0
Superplasticizer                 0
Coarse Aggregate                 0
Fine Aggregate                   0
Age                              0
Concrete compressive strength    0
dtype: int64

### Train Test Split

In [ ]:
concrete_df_test = concrete_df.iloc[501:631]
print(f"test sample size: {len(concrete_df_test)}", concrete_df_test.head(), '\n\n')
concrete_df_train = concrete_df.drop(concrete_df_test.index)
print(f"train sample size: {len(concrete_df_train)}", concrete_df_train.head())

test sample size: 130      Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
501   491.0                26.0    123.0  210.0               3.9   
502   491.0                26.0    123.0  210.0               3.9   
503   491.0                26.0    123.0  210.0               3.9   
504   491.0                26.0    123.0  201.0               3.9   
505   491.0                26.0    123.0  201.0               3.9   

     Coarse Aggregate  Fine Aggregate  Age  Concrete compressive strength  
501             882.0           699.0    3                          25.61  
502             882.0           699.0    7                          33.49  
503             882.0           699.0   56                          59.59  
504             822.0           699.0    3                          29.55  
505             822.0           699.0    7                          37.92   


train sample size: 900    Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0   540.0     

### Define Standarize Function (MinMax)

In [ ]:
def norm_data(df, train_df=None):
  if train_df is not None:
    return (df - train_df.min()) / (train_df.max() - train_df.min())
  else:
    return (df - df.min()) / (df.max() - df.min()).replace(0, 1)

# Call multivariate linear regression

In [ ]:
X_train_raw = concrete_df_train.drop(columns=["Concrete compressive strength"])
y_train_raw = concrete_df_train["Concrete compressive strength"]


X_test_raw = concrete_df_test.drop(columns=["Concrete compressive strength"])
y_test_raw = concrete_df_test["Concrete compressive strength"]

### Add Bias term

In [ ]:
X_train_const = sm.add_constant(X_train_raw)
X_test_const = sm.add_constant(X_test_raw)

# Fit OLS

In [ ]:
model = sm.OLS(y_train_raw, X_train_const)
results = model.fit()

print(results.summary())

                                  OLS Regression Results                                 
Dep. Variable:     Concrete compressive strength   R-squared:                       0.624
Model:                                       OLS   Adj. R-squared:                  0.621
Method:                            Least Squares   F-statistic:                     184.9
Date:                           Sun, 12 Oct 2025   Prob (F-statistic):          1.80e-183
Time:                                   17:06:47   Log-Likelihood:                -3367.7
No. Observations:                            900   AIC:                             6753.
Df Residuals:                                891   BIC:                             6797.
Df Model:                                      8                                         
Covariance Type:                       nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
-------------

In [ ]:
y_pred = results.predict(X_test_const)

from sklearn.metrics import mean_squared_error, r2_score
print("Test MSE:", mean_squared_error(targets_test, y_pred))
print("Test R²:", r2_score(targets_test, y_pred))